# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# import libraries
import re 
import pandas as pd 
import numpy as np 
from sqlalchemy import create_engine

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [3]:
engine = create_engine('sqlite:///DisasterResponseProject.db')
df = pd.read_sql("SELECT * FROM Message", engine)

In [4]:
# Create a function to load data from database
def load_data(): 
    engine = create_engine('sqlite:///DisasterResponseProject.db')
    df = pd.read_sql("SELECT * FROM Message", engine)
    
    #load and view the text and category variables
    X = df.message.values
    y = df.iloc[:, 4:].values
    
    #get categories/column names
    categories = df.columns[4:].tolist()
    
    return X, y, categories

In [5]:
X, y, categories= load_data()
X.shape, y.shape

((26216,), (26216, 35))

### 2. Write a tokenization function to process your text data

In [6]:
#Create a function that normalizes, tokenizes and lemmatizes the messages
def tokenize(text):
    #remove punctuation such as Twitter(#) and tags(@) and convert to lowercase
    text = re.sub(r"[^a-zA-Z0-9]", " ", text).lower()
    
    #Split text into words using NLTK
    tokens = word_tokenize(text) 
    
    #Initialize lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    #Remove stopwords and lemmatize
    clean_tokens = [
        lemmatizer.lemmatize(w) for w in tokens if w not in stopwords.words("english")
    ]
    
    return clean_tokens

In [7]:
#test out function
for message in X[:5]:
    tokens = tokenize(message)
    print("Original message:", message)
    print("Tokenized:", tokens, "\n")

Original message: Weather update - a cold front from Cuba that could pass over Haiti
Tokenized: ['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti'] 

Original message: Is the Hurricane over or is it not over
Tokenized: ['hurricane'] 

Original message: Looking for someone but no name
Tokenized: ['looking', 'someone', 'name'] 

Original message: UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
Tokenized: ['un', 'report', 'leogane', '80', '90', 'destroyed', 'hospital', 'st', 'croix', 'functioning', 'need', 'supply', 'desperately'] 

Original message: says: west side of Haiti, rest of the country today and tonight
Tokenized: ['say', 'west', 'side', 'haiti', 'rest', 'country', 'today', 'tonight'] 



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
def ml_pipeline(): 
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)), 
        ('tfidf', TfidfTransformer()), 
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])
    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
#Split tada into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
#train classifier
pipeline = ml_pipeline()
pipeline.fit(X_train, y_train)

#predict on test data (transofrm test data)
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
print(classification_report(y_test, y_pred, target_names=categories))

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
pipeline.get_params()

In [ ]:
def ml_cv_pipeline(): 
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize, ngram_range=(1, 1))), 
        ('tfidf', TfidfTransformer()), 
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_jobs=-1)))
    ])
    
    # Specify the parameters for grid search 
    parameters = {
        # Try different n-gram ranges (1 word or 1-2 words together)
        'vect__ngram_range': [(1, 1)],
        # Try different number of trees in the random forest 
        'clf__estimator__n_estimators': [50, 100]
    }
    
    # Create a grid search object
    cv = GridSearchCV(pipeline, param_grid=parameters, cv=2, n_jobs=-1)
    return cv

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
#train classifier
ml_cv = ml_cv_pipeline()
ml_cv.fit(X_train, y_train)

#Get the best parameters
print('Best parameters found:', ml_cv.best_params_)

#predict on test data (trannsform test data)
y_pred = ml_cv.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred, target_names=categories))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.